In [112]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [113]:
import numpy as np


In [114]:


import os
import matplotlib.pyplot as plt
%matplotlib inline

import pickle
CIFAR_DIR='/content/drive/MyDrive/data/cifar-10-batches-py/'

def unpickle(file):
    
    with open(file,'rb') as fo:
        cifar_dict=pickle.load(fo,encoding='bytes')
    return cifar_dict


dirs=['batches.meta','data_batch_1','data_batch_2','data_batch_3','data_batch_4','data_batch_5','test_batch']

all_data=[0,1,2,3,4,5,6]
for i,direc in zip(all_data,dirs):
    all_data[i]=unpickle(CIFAR_DIR+direc)
meta=all_data[0]
batch1=all_data[1]
batch2=all_data[2]
batch3=all_data[3]
batch4=all_data[4]
batch5=all_data[5]
test_batch=all_data[6]


In [115]:
import matplotlib.pyplot as plt
X=batch1[b"data"]
# (X[0].shape) =(3072,)
# 3072=3*32*32, in which 3 is number of color channel, 32 *32 is image size. For illustration, refer this link:
# https://towardsdatascience.com/cifar-10-image-classification-in-tensorflow-5b501f7dc77c
X=X.reshape(10000,3,32,32).transpose(0,2,3,1).astype("uint8")
Y=batch1[b"labels"]
#  e.g. Y[0] =6

In [116]:
def one_hot_encode(vec,vals=10):
    n=len(vec)
    out=np.zeros((n,vals))
    out[range(n),vec]=1
    return out

#  E.g : print (one_hot_encode(np.array([2,3])))
# [[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
#  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]

In [117]:

# e.g. of np.vstack :
# a=np.array([1,1,1])
# b=np.array([2,2,2])
# c=np.array([3,3,3])
# print (np.vstack([a,b,c]))
# [[1 1 1]
#  [2 2 2]
#  [3 3 3]]

# e.g. of np.hstack
# a = np.array((1,2,3))
# b = np.array((4,5,6))
# np.hstack((a,b))
# array([1, 2, 3, 4, 5, 6])
# a = np.array([[1],[2],[3]])
# b = np.array([[4],[5],[6]])
# np.hstack((a,b))
# array([[1, 4],
#        [2, 5],
#        [3, 6]])
class Helper():
    def __init__(self):

        self.i=0
        #    i is variable used for batch iteration?
        self.all_train_batches=[batch1,batch2,batch3,batch4,batch5]
        self.test_batches=[test_batch]
        self.training_images=None
        self.training_labels=None
        self.test_images=None
        self.test_labels=None
    def set_up(self):
        a=np.array([1,1,1])
        b=np.array([2,2,2])
        c=np.array([3,3,3])
# # Here we reshape and transpose since Convnet receives the input in shape (width, height, color channel) and divide by 225 so that the value is within (0,1)
        
        self.training_images=np.vstack([d[b"data"] for d in self.all_train_batches])
        train_len=len(self.training_images)
        self.training_images=self.training_images.reshape(train_len,3,32,32).transpose(0,2,3,1)/255
        self.training_labels=one_hot_encode(np.hstack([d[b"labels"] for d in self.all_train_batches] ),10)
        
        self.test_images=np.vstack([d[b"data"] for d in self.test_batches])
        train_len=len(self.test_images)
        self.test_images=self.test_images.reshape(train_len,3,32,32).transpose(0,2,3,1)/255
        self.test_labels=one_hot_encode(np.hstack([d[b"labels"] for d in self.test_batches] ),10)
    def next_batch (self,batch_size):
        x=self.training_images[self.i:self.i+batch_size].reshape(100,32,32,3)
        y=self.training_labels[self.i:self.i+batch_size]
        self.i=(self.i+batch_size)%len(self.training_images)
        return x,y

In [118]:
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior() 
helper=Helper()
helper.set_up()

x=tf.placeholder(tf.float32,shape=[None,32,32,3])
y_true=tf.placeholder(tf.float32,shape=[None,10])
hold_prob=tf.placeholder(tf.float32)

In [119]:
def init_weights(shape):
    random_dist=tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(random_dist)
def init_bias(shape):
    init_bias_vals=tf.constant(0.1,shape=shape)
    return tf.Variable(init_bias_vals)
def conv2d(x,W):
    x = tf.cast(x, tf.float32)
#     W  is filter/kernel, which usuallly has shape of 4,
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')
def max_pool_2by2(x):
    x = tf.cast(x, tf.float32)
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
def convolutional_layer(input_x,shape):
    W=init_weights(shape)
#     for convo_1 shape [4,4,3,32], shape b will be 32
# after convolution, we get a*b*C,then activation would be Wx+b would have shape out-channel, here b is 32


    b=init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x,W)+b)
def normal_full_layer(input_layer,size):
#     size is 1024 in case full_layer below, 
    input_size=int(input_layer.get_shape()[1])
#     e.g, input size of convo_2_pooling is 8*8*64
    W=init_weights([input_size,size])
#     shape W :(4096 1024)
    b=init_bias([size])
    return tf.matmul(input_layer,W)+b

In [120]:
# here [4,4,3,32] is shape of kernel
# [4,4 is kernel width and height]
# 3 is kernel channel in
# 32 is kernel channel out
# shape x : (N 32 32 3)

convo_1=convolutional_layer(x,shape=[4,4,3,32])
# shape convo_1: (N 32 32 32)
convo_1_pooling=max_pool_2by2(convo_1)
# shape convo_1: (N 16 16 32)
convo_2=convolutional_layer(convo_1_pooling,shape=[4,4,32,64])
# shape convo_2 : (N 16 16 64)
convo_2_pooling=max_pool_2by2(convo_2)
# shape convo_2_pooling : (N 8 8 64)
#  -1 let the tf calculate the shape for us
convo_2_flat=tf.reshape(convo_2_pooling,[-1,8*8*64])
full_layer_one=tf.nn.relu(normal_full_layer(convo_2_flat,1024))
full_one_dropout=tf.nn.dropout(full_layer_one,keep_prob=hold_prob)
# normal_full_layer  predicts the result in the last layer ,which 10 is number  label 
y_pred=normal_full_layer(full_one_dropout,10)
cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))

optimizer=tf.train.AdamOptimizer(learning_rate=0.0017)
train=optimizer.minimize(cross_entropy)
init=tf.global_variables_initializer()


In [121]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range (500):
        batch=helper.next_batch(100)
    
       
        sess.run(train,feed_dict={x:batch[0],y_true:batch[1],hold_prob:0.5})
        if i%100==0:
            print('on step{}'.format (i))
            print('Accuracy :')
            matches=tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
            acc=tf.reduce_mean(tf.cast(matches,tf.float32))
            print (sess.run(acc,feed_dict={x:helper.test_images,y_true:helper.test_labels,hold_prob:0.5}))
            print ('\n')

on step0
Accuracy :
0.1014


on step100
Accuracy :
0.3716


on step200
Accuracy :
0.4277


on step300
Accuracy :
0.4744


on step400
Accuracy :
0.4942


